In [ ]:
#topic based clustering-abstrative summary 
# based on Parrot project

In [26]:
import keybert
from transformers import pipeline
import tqdm as notebook_tqdm
import pandas as pd
import math

In [4]:
# Initialization of summarizer based on Bart
summarizer = pipeline("summarization", "vmarklynn/bart-large-cnn-samsum-acsi-ami-v2", truncation=True)
kw_model = keybert.KeyBERT(model='all-mpnet-base-v2')

/opt/homebrew/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
def formatText(text):
    formatted_data = [f"{row['speaker_label']}: {row['text']}" for _, row in text.iterrows()]
    formatted_text = "\n".join([f"{line}" for line in formatted_data])
    return formatted_text

In [27]:
def summarizeText(transcirpt):
    
    
    text = formatText(transcirpt)
    
    #print("\n\n", text, "\n\n")
    # print( "min: ", math.ceil(int(wordCount) * 0.1), "max: ", math.ceil(int(wordCount) * 0.25))
    print("\n\nSummarizing...")
    summary = summarizer(text)[0]['summary_text']
    print("\n", summary, "\n")
    
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(1, 1), 
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list_1= list(dict(keywords).keys())
    print(keywords_list_1)
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(2, 2), 
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list_2= list(dict(keywords).keys())
    print(keywords_list_2)    
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(3, 3), 
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)    
    keywords_list_3 = list(dict(keywords).keys())
    print(keywords_list_3)
    
    response = {'transcription': formatText, 'summary': summary, 
                'keywords_list_1': keywords_list_1, 'keywords_list_2': keywords_list_2,
                'keywords_list_3': keywords_list_3,}
    return response

#-----------------------------------------------------------------------------------------------------               

def summarizeSummary(summary_input):
    
    
    wordCount = 1024 #post_data.get('wordCount-summ')
    
    print( "min: ", math.ceil(int(wordCount) * 0.1), "max: ", math.ceil(int(wordCount) * 0.25))
    print("\n\nSummarizing again...")
    summary = summarizer(summary_input, min_length = math.ceil(int(wordCount) * 0.1), max_length = math.ceil(int(wordCount) * 0.25))[0]['summary_text']
    print("\n", summary, "\n")
    
    response = {'summary': summary}
    return response

In [5]:
transcirpt=pd.read_csv('transcript_1.csv')

In [6]:
transcirpt.head()

,speaker_label,start_time,end_time,text
0,spk_0,0.000150,1.190317,I can record and we dont have a ton of items t...
1,spk_1,1.203483,1.265167,I thought we had in Slack sort of farmed each ...
2,spk_0,1.277667,1.901833,"Thats so I guess that the next, so it looks li..."
3,spk_2,1.947317,2.007650,"uh, on Slack only and on the issue actually?"
4,spk_3,2.014333,2.270333,"Yeah, same, not in, not in real time, but, you..."


In [28]:
summary_transcript= summarizeText(transcirpt)
print(summary_transcript['summary'])



Summarizing...

 The meeting was about the progress of the team's work on the C-zero project. The meeting began with a general overview of the last meeting. The team was very satisfied with the progress they had made on the project, but they were still looking for more work to do. After that, the team discussed the future direction of the project. They discussed how the team could further improve the project and how they could make it better for the next meeting. Finally, the meeting ended with a brief discussion about the future of their project. 

['events', 'support', 'announcements', 'sponsors', 'slack']
['events team', 'event support', 'sponsors event', 'corporate events', 'pr team']
['support gtm teams', 'corp events team', 'event support pm', 'sponsors event support', 'event support need']
The meeting was about the progress of the team's work on the C-zero project. The meeting began with a general overview of the last meeting. The team was very satisfied with the progress they

In [29]:
summarized_summary = summarizeSummary(summary_transcript['summary'])

Your max_length is set to 256, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


min:  103 max:  256


Summarizing again...

 The meeting was about the progress of the team's work on the C-zero project. The team was satisfied with the progress they had made on the project, but they were still looking for more work to do. They discussed the future direction of the project and how they could make it better for the next meeting. The meeting ended with a brief discussion about the future of their project, including how they would further improve the project for future meetings and future work on their C- zero project.  

